<a href="https://colab.research.google.com/github/timkabot/NeuralNetworkMusicGenerator/blob/master/Lyrics_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/mousehead/songlyrics
Link to lyrics dataset

In [ ]:
import os
import pandas as pd

%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
try:
  import gpt_2_simple as gpt2
except:
  !pip3 -q install gpt-2-simple
  import gpt_2_simple as gpt2

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Let's load our data. It was saved into a public Github repository to make it more convenient to be read into Colab:

In [ ]:
dfs = []
link = ('https://raw.githubusercontent.com/ThomasVrancken/lyrics_generation/master/songdata_{}.csv')
for i in range(4):
  dfs.append(pd.read_csv(link.format(i)))

df = pd.concat(dfs).reset_index(drop=True)

Then save this file on our VM’s directory, under a “content” folder. GPT-2 will directly load it from there.

In [ ]:
if not os.path.exists('content'):
    os.makedirs('content')

pd.DataFrame({"lyrics": df['text']})\
    .to_csv(os.path.join('content', 'lyrics.csv'), index=False)

Next, we download the GPT-2 models. Note that you can restrict this list to the model(s) you will be using.

In [ ]:
for model_name in ["124M"]:  # Choose from ["124M","355M","774M"]
    gpt2.download_gpt2(model_name=model_name)   # model is saved into current directory under /models/124M/

Fetching checkpoint: 1.05Mit [00:00, 733Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 159Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 842Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:02, 167Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 543Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 290Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 224Mit/s]                                                       


In [ ]:
learning_rate = 0.0001
optimizer = 'adam' # adam or sgd
batch_size = 1
model_name = "124M" # Has to match one downloaded locally
sess = gpt2.start_tf_sess()

Next, we start the fine tuning. It can take a while (e.g. about 22min for 500 steps with the 774M model). Some useful parameters:


*   **restore_from**: Set to fresh to start training from the base GPT-2, or set to latest to restart training from an existing checkpoint
*   **sample_every**: Number of steps to print example output
*   **print_every**: Number of steps to print training progress
*   **learning_rate**: Learning rate for the training. (default 1e-4, can lower to 1e-5 if you have <1MB input data)
*   **run_name**: Subfolder within checkpoint to save the model. This is useful if you want to work with multiple models (will also need to specify run_name when loading the model)
*  **overwrite**: Set to True if you want to continue finetuning an existing model (w/ restore_from='latest') without creating duplicate copies
* **steps**: How many training steps to be performed.




In [ ]:
gpt2.finetune(sess,
              'content/lyrics.csv',
              model_name=model_name,
              sample_every=50,
              print_every=10,
              learning_rate=learning_rate,
              batch_size=batch_size,
              restore_from='latest',
              steps=1000)   # max number of training steps

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  1.17it/s]


dataset has 23199731 tokens
Training...
[10 | 19.89] loss=2.07 avg=2.07
[20 | 32.34] loss=2.19 avg=2.13
[30 | 44.81] loss=2.28 avg=2.18
[40 | 57.27] loss=1.84 avg=2.09
[50 | 69.74] loss=2.03 avg=2.08
======== SAMPLE 1 ========
 tell and I won't be the first time I've gotten a lot of trouble  
From the depths of despair.  
Truly,  
In my last hour as a king,  
I could not believe  
The price I was getting for my love  
I tried harder on all these years  
 But I didn't take the bait.  
Then I was in the middle of her story.  
My heart just kept pounding  
I prayed for her,  
  
I got lost in the mire  
I had no idea what else to do.  
As an hour passed by  
We got lost in the mire  
  
The time was late.  
  
I looked over the sky and saw tears streaming  
 From the depths of despair.  
In my last hour as a king.  
  
In my last hour on the road  
I couldn't believe  
The price I was getting for  
  
I was lost in the mire  
In my last hour as a king.  
  
In my last hour as a king  
I p

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

NOW MODEL IS BORN

In [ ]:
lst_results=gpt2.generate(
    sess,
    nsamples=3,
    temperature=1.5, # change me
    top_p=0.5, # Change me
    return_as_list=True,
    prefix="<|startoftext|>",
    truncate="<|endoftext|>",
    include_prefix=False
    )

for res in lst_results:
  print(res)
  print('\n -------//------ \n')

Blue waves rose and stretched and scattered. There are things that seem sweet to me. There is hope, hope that I believe I can feel. There is love, and so that's why I stay here. Hope is blue! Hope is blue, now is the time to see that!  
Hope's blue. And so is love. It's an everyday part of my day. The evening after all these dreams I hear.  
I dreamt the day in this one small and true blue.  
I said blue wave comes on high,  
And hopes for evening peace and bliss. With love and an abundance of sunshine my whole love can show.  
  
(fade)  
This smile and smile of theirs is true.  
I never doubted that happiness should fall, still more we did believe. There is such life that I love  
All too. I see it coming with all my love to him  
You and me together!  
  
This smile of theirs is true. We lived in love,  
On day ten this time. It was never enough  
And there's such life we're in  
And love and there is like that inside. You  
and me!  
Love me all of the days,  
The hours and the the

In [ ]:
!zip -r /content/model.zip /content/checkpoint


  adding: content/checkpoint/ (stored 0%)
  adding: content/checkpoint/run1/ (stored 0%)
  adding: content/checkpoint/run1/checkpoint (deflated 40%)
  adding: content/checkpoint/run1/hparams.json (deflated 28%)
  adding: content/checkpoint/run1/model-1000.data-00000-of-00001 (deflated 7%)
  adding: content/checkpoint/run1/events.out.tfevents.1585665325.654fa7a1b6ca (deflated 61%)
  adding: content/checkpoint/run1/model-1000.index (deflated 62%)
  adding: content/checkpoint/run1/vocab.bpe (deflated 53%)
  adding: content/checkpoint/run1/encoder.json (deflated 67%)
  adding: content/checkpoint/run1/counter (stored 0%)
  adding: content/checkpoint/run1/model-1000.meta (deflated 91%)
